In [1]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#Load ticker data
tickerdata = pd.read_excel(r'Data/Processed Data for Thesis with New Design Matrix.xlsx',engine='openpyxl',dtype=str)
tickerdata['CI'] = pd.to_numeric(tickerdata['CI'])
tickerdata['ISF'] = pd.to_numeric(tickerdata['ISF'])
tickerdata.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Start(t=1),End(t=1),Start(t=3),...,Start(t=12),End(t=12),SP(Start(t=1)),SP(End(t=1)),SP(Start(t=3)),SP(End(t=3)),SP(Start(t=6)),SP(End(t=6)),SP(Start(t=12)),SP(End(t=12))
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-07-31 00:00:00,2008-08-31 00:00:00,2008-05-31 00:00:00,...,2007-08-31 00:00:00,2008-08-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-08-31 00:00:00,2008-09-30 00:00:00,2008-08-31 00:00:00,...,2008-08-31 00:00:00,2009-08-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,0,2008-07-31 00:00:00,2008-08-31 00:00:00,2008-05-31 00:00:00,...,2007-08-31 00:00:00,2008-08-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,1,2008-08-31 00:00:00,2008-09-30 00:00:00,2008-08-31 00:00:00,...,2008-08-31 00:00:00,2009-08-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bird,Giant Manufacturing Co Ltd,9921,TWSE,1,TAIEX,0,2017-08-30 00:00:00,2017-09-30 00:00:00,2017-06-30 00:00:00,...,2016-09-30 00:00:00,2017-09-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Reshape/melt ticker data to make time an attribute
transformedtickerdata = tickerdata.melt(id_vars=['Sharing Economy Firm', 'Incumbent Firms', 'Ticker',
       'Exchange', 'CI', 'Market Index', 'ISF', 'SP(Start(t=1))', 'SP(End(t=1))', 'SP(Start(t=3))',
       'SP(End(t=3))', 'SP(Start(t=6))', 'SP(End(t=6))', 'SP(Start(t=12))', 'SP(End(t=12))'],var_name='t',value_name='Date').drop(columns=['SP(Start(t=1))', 'SP(End(t=1))', 'SP(Start(t=3))',
       'SP(End(t=3))', 'SP(Start(t=6))', 'SP(End(t=6))', 'SP(Start(t=12))', 'SP(End(t=12))'])
transformedtickerdata['Ticker'] = transformedtickerdata['Ticker'].astype(str)
transformedtickerdata.tail()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,t,Date
1051,Facebook,Disney,DIS,NYSE,0,NYA,1,End(t=12),2005-02-28 00:00:00
1052,Twitter,Viacom,VIAC,NASDAQ,1,IXIC,0,End(t=12),2006-03-31 00:00:00
1053,Twitter,Viacom,VIAC,NASDAQ,1,IXIC,1,End(t=12),2007-03-31 00:00:00
1054,Twitter,Thomson Reuters Corp,TRI,NYSE,0,NYA,0,End(t=12),2006-03-31 00:00:00
1055,Twitter,Thomson Reuters Corp,TRI,NYSE,0,NYA,1,End(t=12),2007-03-31 00:00:00


In [4]:
#Load equity data and transform
equitydata = pd.read_excel(r'Data/Bloomberg Equity Data.xlsx', engine='openpyxl',usecols=[0,1,2],sheet_name=None,dtype=str)
datalist = []
for ticker, data in equitydata.items():
    data['Ticker'] = str(ticker)
    datalist.append(data)
transformedequitydata = pd.concat(datalist,axis=0)
transformedequitydata.dropna(axis=1,how='all')
transformedequitydata['Last Price'] = pd.to_numeric(transformedequitydata['Last Price'])
transformedequitydata.head()

,Date,Last Price,Volume,Ticker
0,2009-08-31 00:00:00,22.4513,3111839,MAR
1,2009-08-30 00:00:00,23.0337,4564532,MAR
2,2009-08-29 00:00:00,23.0337,4564532,MAR
3,2009-08-28 00:00:00,23.0337,4564532,MAR
4,2009-08-27 00:00:00,22.9022,4047335,MAR


In [5]:
#Load index data and transform
# indexdata = pd.read_excel(r'Bloomberg Index Data.xlsx', engine='openpyxl',usecols=[0,1],sheet_name=None,dtype=str)
# indexlist = []
# for index, data in indexdata.items():
#     data['Index'] = str(index)
#     indexlist.append(data)
# transformedindexdata = pd.concat(indexlist,axis=0)
# transformedindexdata['Last Price'] = pd.to_numeric(transformedindexdata['Last Price'])
# transformedindexdata.head()

In [6]:
#Merge ticker and equity data
merged_data = transformedtickerdata.merge(transformedequitydata, how='left',left_on=['Date','Ticker'],right_on=['Date','Ticker']).dropna(axis=1,how='all').rename(columns={'Last Price':'SP(t)','Volume':'Volume(t)'})
merged_data['Volume(t)'] = pd.to_numeric(merged_data['Volume(t)'])
merged_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,t,Date,SP(t),Volume(t)
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,Start(t=1),2008-07-31 00:00:00,24.15840,3808057
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,Start(t=1),2008-08-31 00:00:00,26.30290,2756931
2,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,0,Start(t=1),2008-07-31 00:00:00,45.52000,5017749
3,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,1,Start(t=1),2008-08-31 00:00:00,47.41000,3567702
4,Bird,Giant Manufacturing Co Ltd,9921,TWSE,1,TAIEX,0,Start(t=1),2017-08-30 00:00:00,5.08187,1008183


In [7]:
#Merge index data with ticker/equity data
# merged_data=merged_data.merge(transformedindexdata,how='left',left_on=['Date','Market Index'],right_on=['Date','Index']).drop(columns='Index').rename(columns={'Last Price':'Index Level'})
# merged_data.head()

In [8]:
#Re-Pivot Data
complete_data = merged_data.pivot(index=['Sharing Economy Firm','Incumbent Firms','Ticker','Exchange','CI','Market Index','ISF'],columns=['t'])
complete_data.head()

Date  \
t                                                                                             End(t=1)   
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2008-08-31 00:00:00   
                                                                              1    2008-09-30 00:00:00   
                     Marriott International   MAR    NASDAQ   1  IXIC         0    2008-08-31 00:00:00   
                                                                              1    2008-09-30 00:00:00   
Bird                 Fox Factory Holding Corp FOXF   NASDAQ   0  IXIC         0    2017-09-30 00:00:00   

                                                                                                        \
t                                                                                            End(t=12)   
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2008-08-31 00:00:00   
                                                                              1    2009-08-31 00:00:00   
                     Marriott International   MAR    NASDAQ   1  IXIC         0    2008-08-31 00:00:00   
                                                                              1    2009-08-31 00:00:00   
Bird                 Fox Factory Holding Corp FOXF   NASDAQ   0  IXIC         0    2017-09-30 00:00:00   

                                                                                                        \
t                                                                                             End(t=3)   
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2008-08-31 00:00:00   
                                                                              1    2008-11-30 00:00:00   
                     Marriott International   MAR    NASDAQ   1  IXIC         0    2008-08-31 00:00:00   
                                                                              1    2008-11-30 00:00:00   
Bird                 Fox Factory Holding Corp FOXF   NASDAQ   0  IXIC         0    2017-09-30 00:00:00   

                                                                                                        \
t                                                                                             End(t=6)   
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2008-08-31 00:00:00   
                                                                              1    2009-02-28 00:00:00   
                     Marriott International   MAR    NASDAQ   1  IXIC         0    2008-08-31 00:00:00   
                                                                              1    2009-02-28 00:00:00   
Bird                 Fox Factory Holding Corp FOXF   NASDAQ   0  IXIC         0    2017-09-30 00:00:00   

                                                                                                        \
t                                                                                           Start(t=1)   
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2008-07-31 00:00:00   
                                                                              1    2008-08-31 00:00:00   
                     Marriott International   MAR    NASDAQ   1  IXIC         0    2008-07-31 00:00:00   
                                                                  

In [9]:
#Drop Multindex
complete_data.columns = ['_'.join(col) for col in complete_data.columns]
complete_data.head()

Date_End(t=1)  \
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2008-08-31 00:00:00   
                                                                              1    2008-09-30 00:00:00   
                     Marriott International   MAR    NASDAQ   1  IXIC         0    2008-08-31 00:00:00   
                                                                              1    2008-09-30 00:00:00   
Bird                 Fox Factory Holding Corp FOXF   NASDAQ   0  IXIC         0    2017-09-30 00:00:00   

                                                                                        Date_End(t=12)  \
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2008-08-31 00:00:00   
                                                                              1    2009-08-31 00:00:00   
                     Marriott International   MAR    NASDAQ   1  IXIC         0    2008-08-31 00:00:00   
                                                                              1    2009-08-31 00:00:00   
Bird                 Fox Factory Holding Corp FOXF   NASDAQ   0  IXIC         0    2017-09-30 00:00:00   

                                                                                         Date_End(t=3)  \
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2008-08-31 00:00:00   
                                                                              1    2008-11-30 00:00:00   
                     Marriott International   MAR    NASDAQ   1  IXIC         0    2008-08-31 00:00:00   
                                                                              1    2008-11-30 00:00:00   
Bird                 Fox Factory Holding Corp FOXF   NASDAQ   0  IXIC         0    2017-09-30 00:00:00   

                                                                                         Date_End(t=6)  \
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2008-08-31 00:00:00   
                                                                              1    2009-02-28 00:00:00   
                     Marriott International   MAR    NASDAQ   1  IXIC         0    2008-08-31 00:00:00   
                                                                              1    2009-02-28 00:00:00   
Bird                 Fox Factory Holding Corp FOXF   NASDAQ   0  IXIC         0    2017-09-30 00:00:00   

                                                                                       Date_Start(t=1)  \
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2008-07-31 00:00:00   
                                                                              1    2008-08-31 00:00:00   
                     Marriott International   MAR    NASDAQ   1  IXIC         0    2008-07-31 00:00:00   
                                                                              1    2008-08-31 00:00:00   
Bird                 Fox Factory Holding Corp FOXF   NASDAQ   0  IXIC         0    2017-08-30 00:00:00   

                                                                                      Date_Start(t=12)  \
Sharing Economy Firm Incumbent Firms          Ticker Exchange CI Market Index ISF                        
Airbnb               Las Vegas Sands          LVS    NYSE     0  NYA          0    2007-08-31 00:00:00   
                                                        

In [10]:
#Print column names to drop and rename
complete_data.columns

Index(['Date_End(t=1)', 'Date_End(t=12)', 'Date_End(t=3)', 'Date_End(t=6)',
       'Date_Start(t=1)', 'Date_Start(t=12)', 'Date_Start(t=3)',
       'Date_Start(t=6)', 'SP(t)_End(t=1)', 'SP(t)_End(t=12)',
       'SP(t)_End(t=3)', 'SP(t)_End(t=6)', 'SP(t)_Start(t=1)',
       'SP(t)_Start(t=12)', 'SP(t)_Start(t=3)', 'SP(t)_Start(t=6)',
       'Volume(t)_End(t=1)', 'Volume(t)_End(t=12)', 'Volume(t)_End(t=3)',
       'Volume(t)_End(t=6)', 'Volume(t)_Start(t=1)', 'Volume(t)_Start(t=12)',
       'Volume(t)_Start(t=3)', 'Volume(t)_Start(t=6)'],
      dtype='object')

In [11]:
#Drop Date Columns/Reset Index.
complete_data = complete_data.drop(columns=['Date_Start(t=1)', 'Date_Start(t=12)', 'Date_Start(t=3)', 'Date_Start(t=6)']).reset_index().rename(columns={'SP(t)_End(t=1)':'SP_End(t=1)','SP(t)_End(t=3)':'SP_End(t=3)','SP(t)_End(t=6)':'SP_End(t=6)','SP(t)_End(t=12)':'SP_End(t=12)','SP(t)_Start(t=1)':'SP_Start(t=1)','SP(t)_Start(t=3)':'SP_Start(t=3)','SP(t)_Start(t=6)':'SP_Start(t=6)','SP(t)_Start(t=12)':'SP_Start(t=12)','Volume(t)_End(t=1)':'Volume_End(t=1)','Volume(t)_End(t=3)':'Volume_End(t=3)','Volume(t)_End(t=6)':'Volume_End(t=6)','Volume(t)_End(t=12)':'Volume_End(t=12)','Volume(t)_Start(t=1)':'Volume_Start(t=1)','Volume(t)_Start(t=3)':'Volume_Start(t=3)','Volume(t)_Start(t=6)':'Volume_Start(t=6)','Volume(t)_Start(t=12)':'Volume_Start(t=12)'})
# , 'Index Level_t':'Index Level(t)', 'Index Level_t+1':'Index Level(t+1)','Index Level_t+12':'Index Level(t+12)', 'Index Level_t+3':'Index Level(t+3)', 'Index Level_t+6':'Index Level(t+6)','Index Level_t-1':'Index Level(t-1)', 'Index Level_t-12':'Index Level(t-12)', 'Index Level_t-3':'Index Level(t-3)','Index Level_t-6':'Index Level(t-6)'
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Date_End(t=1),Date_End(t=12),Date_End(t=3),...,SP_Start(t=3),SP_Start(t=6),Volume_End(t=1),Volume_End(t=12),Volume_End(t=3),Volume_End(t=6),Volume_Start(t=1),Volume_Start(t=12),Volume_Start(t=3),Volume_Start(t=6)
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,0,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,...,69.4400,83.3000,3567702,3567702,3567702,3567702,5017749,2489400,2883924,2853416
1,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,1,2008-09-30 00:00:00,2009-08-31 00:00:00,2008-11-30 00:00:00,...,47.4100,47.4100,2854762,22988568,12411226,28421998,3567702,3567702,3567702,3567702
2,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,...,30.6852,31.7947,2756931,2756931,2756931,2756931,3808057,1691184,3861316,3165477
3,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-09-30 00:00:00,2009-08-31 00:00:00,2008-11-30 00:00:00,...,26.3029,26.3029,8810031,3111839,1496529,6908523,2756931,2756931,2756931,2756931
4,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,IXIC,0,2017-09-30 00:00:00,2017-09-30 00:00:00,2017-09-30 00:00:00,...,35.6000,28.6000,316880,316880,316880,316880,332831,342422,412887,372828


In [12]:
#Add Columns for Share Price Growth Rate % for t=(1,3,6,12)
complete_data['% Change in Share Price (t=1)'] = ((complete_data['SP_End(t=1)']-complete_data['SP_Start(t=1)'])/complete_data['SP_Start(t=1)'])*100
complete_data['% Change in Share Price (t=3)'] = ((complete_data['SP_End(t=3)']-complete_data['SP_Start(t=3)'])/complete_data['SP_Start(t=3)'])*100
complete_data['% Change in Share Price (t=6)'] = ((complete_data['SP_End(t=6)']-complete_data['SP_Start(t=6)'])/complete_data['SP_Start(t=6)'])*100
complete_data['% Change in Share Price (t=12)'] = ((complete_data['SP_End(t=12)']-complete_data['SP_Start(t=12)'])/complete_data['SP_Start(t=12)'])*100
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Date_End(t=1),Date_End(t=12),Date_End(t=3),...,Volume_End(t=3),Volume_End(t=6),Volume_Start(t=1),Volume_Start(t=12),Volume_Start(t=3),Volume_Start(t=6),% Change in Share Price (t=1),% Change in Share Price (t=3),% Change in Share Price (t=6),% Change in Share Price (t=12)
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,0,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,...,3567702,3567702,5017749,2489400,2883924,2853416,4.152021,-31.725230,-43.085234,-52.447342
1,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,1,2008-09-30 00:00:00,2009-08-31 00:00:00,2008-11-30 00:00:00,...,12411226,28421998,3567702,3567702,3567702,3567702,-23.834634,-89.095128,-95.190888,-69.921957
2,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,...,2756931,2756931,3808057,1691184,3861316,3165477,8.876830,-14.281478,-17.272690,-36.492656
3,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-09-30 00:00:00,2009-08-31 00:00:00,2008-11-30 00:00:00,...,1496529,6908523,2756931,2756931,2756931,2756931,-7.515141,-40.482228,-49.805155,-14.643252
4,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,IXIC,0,2017-09-30 00:00:00,2017-09-30 00:00:00,2017-09-30 00:00:00,...,316880,316880,332831,342422,412887,372828,8.291457,21.067416,50.699301,87.636047


In [13]:
#Add Columns for Volume Growth Rate % for t=(1,3,6,12)
complete_data['% Change in Volume (t=1)'] = ((complete_data['Volume_End(t=1)']-complete_data['Volume_Start(t=1)'])/complete_data['Volume_Start(t=1)'])*100
complete_data['% Change in Volume (t=3)'] = ((complete_data['Volume_End(t=3)']-complete_data['Volume_Start(t=3)'])/complete_data['Volume_Start(t=3)'])*100
complete_data['% Change in Volume (t=6)'] = ((complete_data['Volume_End(t=6)']-complete_data['Volume_Start(t=6)'])/complete_data['Volume_Start(t=6)'])*100
complete_data['% Change in Volume (t=12)'] = ((complete_data['Volume_End(t=12)']-complete_data['Volume_Start(t=12)'])/complete_data['Volume_Start(t=12)'])*100
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Date_End(t=1),Date_End(t=12),Date_End(t=3),...,Volume_Start(t=3),Volume_Start(t=6),% Change in Share Price (t=1),% Change in Share Price (t=3),% Change in Share Price (t=6),% Change in Share Price (t=12),% Change in Volume (t=1),% Change in Volume (t=3),% Change in Volume (t=6),% Change in Volume (t=12)
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,0,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,...,2883924,2853416,4.152021,-31.725230,-43.085234,-52.447342,-28.898357,23.709987,25.032663,43.315739
1,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,1,2008-09-30 00:00:00,2009-08-31 00:00:00,2008-11-30 00:00:00,...,3567702,3567702,-23.834634,-89.095128,-95.190888,-69.921957,-19.983171,247.877317,696.647198,544.352247
2,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,...,3861316,3165477,8.876830,-14.281478,-17.272690,-36.492656,-27.602686,-28.601259,-12.906301,63.017803
3,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-09-30 00:00:00,2009-08-31 00:00:00,2008-11-30 00:00:00,...,2756931,2756931,-7.515141,-40.482228,-49.805155,-14.643252,219.559358,-45.717575,150.587447,12.873300
4,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,IXIC,0,2017-09-30 00:00:00,2017-09-30 00:00:00,2017-09-30 00:00:00,...,412887,372828,8.291457,21.067416,50.699301,87.636047,-4.792522,-23.252609,-15.006384,-7.459217


In [14]:
# Add Columns for Index Level Growth Rate % for t=(1,3,6,12)
# complete_data['% Change in Index Level (t=1)'] = ((complete_data['Index Level_End(t=1)']-complete_data['Index Level_Start(t=1)'])/complete_data['Index Level_Start(t=1)'])*100
# complete_data['% Change in Index Level (t=3)'] = ((complete_data['Index Level_End(t=3)']-complete_data['Index Level_Start(t=3)'])/complete_data['Index Level_Start(t=3)'])*100
# complete_data['% Change in Index Level (t=6)'] = ((complete_data['Index Level_End(t=6)']-complete_data['Index Level_Start(t=6)'])/complete_data['Index Level_Start(t=6)'])*100
# complete_data['% Change in Index Level (t=12)'] = ((complete_data['Index Level_End(t=12)']-complete_data['Index Level_Start(t=12)'])/complete_data['Index Level_Start(t=12)'])*100
# complete_data.head()

In [15]:
#Convert Exchange to Dummy Variables for either NYSE, NASDAQ, or Other (OTCMKTS, TWSE, and NSE)
exchange_dummies = pd.get_dummies(complete_data['Exchange'])
complete_data = pd.concat([complete_data,exchange_dummies],axis=1)
complete_data['Other'] = complete_data['NSE']+complete_data['OTCMKTS']+complete_data['TWSE']
complete_data['Sharing Firm Introduced'] = np.where(complete_data['ISF'] == 1, "Yes", "No")
complete_data['Relatedness with Sharing Firm'] = np.where(complete_data['CI'] == 1, "Closely Related", "Less Closely Related")
complete_data = complete_data.drop(columns=['NSE','OTCMKTS','TWSE']).sort_values(by=['Sharing Economy Firm','CI'],ascending=[True, False])
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Date_End(t=1),Date_End(t=12),Date_End(t=3),...,% Change in Share Price (t=12),% Change in Volume (t=1),% Change in Volume (t=3),% Change in Volume (t=6),% Change in Volume (t=12),NASDAQ,NYSE,Other,Sharing Firm Introduced,Relatedness with Sharing Firm
2,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,...,-36.492656,-27.602686,-28.601259,-12.906301,63.017803,1,0,0,No,Closely Related
3,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-09-30 00:00:00,2009-08-31 00:00:00,2008-11-30 00:00:00,...,-14.643252,219.559358,-45.717575,150.587447,12.873300,1,0,0,Yes,Closely Related
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,0,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,...,-52.447342,-28.898357,23.709987,25.032663,43.315739,0,1,0,No,Less Closely Related
1,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,1,2008-09-30 00:00:00,2009-08-31 00:00:00,2008-11-30 00:00:00,...,-69.921957,-19.983171,247.877317,696.647198,544.352247,0,1,0,Yes,Less Closely Related
6,Bird,Giant Manufacturing Co Ltd,9921,TWSE,1,TAIEX,0,2017-09-30 00:00:00,2017-09-30 00:00:00,2017-09-30 00:00:00,...,-33.146868,-96.741663,-94.926790,-98.501300,-94.113737,0,0,1,No,Closely Related


In [16]:
#Write complete data to Excel and CSV
with pd.ExcelWriter('Data/complete_data.xlsx',engine='openpyxl') as writer:
    complete_data.to_excel(writer)
complete_data.to_csv('Data/complete_data.csv')

In [17]:
#Create STATA ready data
stata_data = complete_data[['CI', 'ISF', 'NASDAQ', 'NYSE', 'Other', '% Change in Volume (t=1)', '% Change in Volume (t=3)', '% Change in Volume (t=6)', '% Change in Volume (t=12)', '% Change in Share Price (t=1)', '% Change in Share Price (t=3)', '% Change in Share Price (t=6)', '% Change in Share Price (t=12)', 'Sharing Firm Introduced','Relatedness with Sharing Firm', 'Date_End(t=1)', 'Date_End(t=12)', 'Date_End(t=3)', 'Date_End(t=6)']].rename(columns={'% Change in Volume (t=1)':'one_m_diff_volume', '% Change in Volume (t=3)':'three_m_diff_volume', '% Change in Volume (t=6)':'six_m_diff_volume', '% Change in Volume (t=12)':'one_y_diff_volume', '% Change in Share Price (t=1)':'one_m_diff_shareprice', '% Change in Share Price (t=3)':'three_m_diff_shareprice', '% Change in Share Price (t=6)':'six_m_diff_shareprice', '% Change in Share Price (t=12)':'one_y_diff_shareprice'})
stata_data['Exchange'] = stata_data[['NASDAQ','NYSE','Other']].idxmax(axis=1)
# '% Change in Growth Rate of Index Level (t=1)', '% Change in Growth Rate of Index Level (t=3)','% Change in Growth Rate of Index Level (t=6)', '% Change in Growth Rate of Index Level (t=12)',
# '% Change in Growth Rate of Index Level (t=1)':'one_m_diff_indexlevel', '% Change in Growth Rate of Index Level (t=3)':'three_m_diff_indexlevel','% Change in Growth Rate of Index Level (t=6)':'six_m_diff_indexlevel', '% Change in Growth Rate of Index Level (t=12)':'one_y_diff_indexlevel',
stata_data.head()

,CI,ISF,NASDAQ,NYSE,Other,one_m_diff_volume,three_m_diff_volume,six_m_diff_volume,one_y_diff_volume,one_m_diff_shareprice,three_m_diff_shareprice,six_m_diff_shareprice,one_y_diff_shareprice,Sharing Firm Introduced,Relatedness with Sharing Firm,Date_End(t=1),Date_End(t=12),Date_End(t=3),Date_End(t=6),Exchange
2,1,0,1,0,0,-27.602686,-28.601259,-12.906301,63.017803,8.876830,-14.281478,-17.272690,-36.492656,No,Closely Related,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,NASDAQ
3,1,1,1,0,0,219.559358,-45.717575,150.587447,12.873300,-7.515141,-40.482228,-49.805155,-14.643252,Yes,Closely Related,2008-09-30 00:00:00,2009-08-31 00:00:00,2008-11-30 00:00:00,2009-02-28 00:00:00,NASDAQ
0,0,0,0,1,0,-28.898357,23.709987,25.032663,43.315739,4.152021,-31.725230,-43.085234,-52.447342,No,Less Closely Related,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,2008-08-31 00:00:00,NYSE
1,0,1,0,1,0,-19.983171,247.877317,696.647198,544.352247,-23.834634,-89.095128,-95.190888,-69.921957,Yes,Less Closely Related,2008-09-30 00:00:00,2009-08-31 00:00:00,2008-11-30 00:00:00,2009-02-28 00:00:00,NYSE
6,1,0,0,0,1,-96.741663,-94.926790,-98.501300,-94.113737,-7.085187,-17.569188,-20.093245,-33.146868,No,Closely Related,2017-09-30 00:00:00,2017-09-30 00:00:00,2017-09-30 00:00:00,2017-09-30 00:00:00,Other


In [18]:
#Write STATA data to Excel and CSV
with pd.ExcelWriter('Data/stata_data.xlsx',engine='openpyxl') as writer:
    stata_data.to_excel(writer)
stata_data.to_csv('Data/stata_data.csv')